In [1]:
import pandas as pd
import pickle 

import pandas as pd
import statistics as myStats
import glob
import numpy as np
import seaborn as sb
from matplotlib import pyplot as plt

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Revome scientific notation to expand numbers in the dataset up to the
# second decimal place.
pd.set_option('display.float_format', lambda x: '%.4f' % x)
#pd.set_option('display.float_format', lambda x: '%,g' % x)

data=pd.read_pickle('all_years.pkl')


In [2]:
data['Sale_Date'] = pd.to_datetime(data['Sale_Date'],)

data = data.sort_values(by="Sale_Date")
#data

In [3]:
yearDataFile = pd.DataFrame(data=data)
yearDataFile['UniSA_Customer_No'] = ['%.12g' % n for n in yearDataFile['UniSA_Customer_No']]
del yearDataFile['Barcode_Item']
del yearDataFile['Receipt_Line_No']
del yearDataFile['Total_Sale_Amount_InclusiveGST']
del yearDataFile['Item_Description']
del yearDataFile['Department_Name']
del yearDataFile['Commodity_Name']
del yearDataFile['Total_Receipt_Lines']
del yearDataFile['Quantity_Sold']
del yearDataFile['Offer']
yearDataFile

,Sale_Date,Sale_Time,UniSA_Receipt_No1,UniSA_Customer_No,Item_Value
100075,2013-01-02,15:14,525539,2690797360,9.9900
100701,2013-01-02,15:23,525542,4498461089,5.4900
100702,2013-01-02,15:23,525542,4498461089,5.4900
100703,2013-01-02,15:23,525542,4498461089,2.0200
100704,2013-01-02,15:23,525542,4498461089,3.1900
...,...,...,...,...,...
12186915,2015-12-31,09:45:46,15123111044042,9413990620,1.5700
12186914,2015-12-31,09:45:46,15123111044042,9413990620,1.4900
12186913,2015-12-31,09:45:46,15123111044042,9413990620,1.4200
12186887,2015-12-31,09:40:48,15123111044041,8548751290,1.4900


In [4]:
# Create a "TimeStamp" variable.

pd.options.mode.chained_assignment = None  # default='warn'

yearDataFile['Sale_Date'].astype(str)
yearDataFile["Sale_Time"][yearDataFile["Sale_Time"] == 'nan'] = "00:00:00"

In [5]:
yearDataFile['Sale_Time'] = yearDataFile['Sale_Time'].astype(str)
yearDataFile['Sale_Time_Length'] = yearDataFile['Sale_Time'].astype(str).map(len)

In [6]:
pd.options.mode.chained_assignment = None  # default='warn'
Seconds = ':00'
yearDataFile["Sale_Time"][yearDataFile["Sale_Time_Length"] == 5] = yearDataFile["Sale_Time"] + Seconds

In [7]:
#Creating a new variable "Time Stamp"

yearDataFile['Sale_Date'] = yearDataFile['Sale_Date'].astype(str)
yearDataFile['TimeStamp'] = yearDataFile["Sale_Date"] + " " + yearDataFile["Sale_Time"]
#yearDataFile

In [8]:
# Generating a "Key" to find number of trips.

yearDataFile['UniSA_Receipt_No1'] = yearDataFile['UniSA_Receipt_No1'].astype(str)
yearDataFile['Key'] = yearDataFile['UniSA_Receipt_No1'] + yearDataFile['TimeStamp']
yearDataFile.drop('Sale_Time_Length', axis=1, inplace=True)
yearDataFile.drop('TimeStamp', axis=1, inplace=True)
#yearDataFile

In [10]:
# Download the file 

#yearDataFile.to_pickle("Modified_Data.pkl")

#df=pd.read_pickle('Modified_data.pkl')

In [11]:
# Convert the 'Sale_Date' to date-time format from string.

yearDataFile['Sale_Date'] = pd.to_datetime(yearDataFile['Sale_Date'],)

In [13]:
# Transform the "Sale Date" column into three different columns, one each for "Sale Day", "Sale Month" and "Sale Year".

yearDataFile['Year'] = pd.DatetimeIndex(yearDataFile['Sale_Date']).year

In [15]:
# Finding week number.

yearDataFile['Week'] = yearDataFile['Sale_Date'].dt.week
yearDataFile

,Sale_Date,Sale_Time,UniSA_Receipt_No1,UniSA_Customer_No,Item_Value,Key,Year,Week
100075,2013-01-02,15:14:00,525539,2690797360,9.9900,5255392013-01-02 15:14:00,2013,1
100701,2013-01-02,15:23:00,525542,4498461089,5.4900,5255422013-01-02 15:23:00,2013,1
100702,2013-01-02,15:23:00,525542,4498461089,5.4900,5255422013-01-02 15:23:00,2013,1
100703,2013-01-02,15:23:00,525542,4498461089,2.0200,5255422013-01-02 15:23:00,2013,1
100704,2013-01-02,15:23:00,525542,4498461089,3.1900,5255422013-01-02 15:23:00,2013,1
...,...,...,...,...,...,...,...,...
12186915,2015-12-31,09:45:46,15123111044042,9413990620,1.5700,151231110440422015-12-31 09:45:46,2015,53
12186914,2015-12-31,09:45:46,15123111044042,9413990620,1.4900,151231110440422015-12-31 09:45:46,2015,53
12186913,2015-12-31,09:45:46,15123111044042,9413990620,1.4200,151231110440422015-12-31 09:45:46,2015,53
12186887,2015-12-31,09:40:48,15123111044041,8548751290,1.4900,151231110440412015-12-31 09:40:48,2015,53


In [26]:
# Split data by years

df2013 = yearDataFile[yearDataFile['Year'] == 2013]
df2014 = yearDataFile[yearDataFile['Year'] == 2014]
df2015 = yearDataFile[yearDataFile['Year'] == 2015]
#df2015

,Sale_Date,Sale_Time,UniSA_Receipt_No1,UniSA_Customer_No,Item_Value,Key,Year,Week
45607,2015-01-02,16:03:18,44568,5291390841,3.9800,445682015-01-02 16:03:18,2015,1
45610,2015-01-02,16:05:05,44569,6078351501,3.2900,445692015-01-02 16:05:05,2015,1
45609,2015-01-02,16:05:05,44569,6078351501,7.2900,445692015-01-02 16:05:05,2015,1
45608,2015-01-02,16:05:05,44569,6078351501,2.4900,445692015-01-02 16:05:05,2015,1
45632,2015-01-02,16:05:05,44569,6078351501,2.3100,445692015-01-02 16:05:05,2015,1
...,...,...,...,...,...,...,...,...
12186915,2015-12-31,09:45:46,15123111044042,9413990620,1.5700,151231110440422015-12-31 09:45:46,2015,53
12186914,2015-12-31,09:45:46,15123111044042,9413990620,1.4900,151231110440422015-12-31 09:45:46,2015,53
12186913,2015-12-31,09:45:46,15123111044042,9413990620,1.4200,151231110440422015-12-31 09:45:46,2015,53
12186887,2015-12-31,09:40:48,15123111044041,8548751290,1.4900,151231110440412015-12-31 09:40:48,2015,53


In [19]:
# First aggregate the 2013 data by week.


aggregations1 = {
   "Item_Value": "sum",
   "Key": pd.Series.nunique
}

GroupWeek2013 = df2013.groupby(by = ["Week"], as_index = False).agg(aggregations1).rename(columns = {"Item_Value": "Weekly_Total_Sales", "Key": "Weekly_Total_Trips"})
GroupWeek2013

,Week,Weekly_Total_Sales,Weekly_Total_Trips
0,1,1069091.6600,19645
1,2,920332.1500,18371
2,3,940726.4600,18176
3,4,978813.5100,18724
4,5,942373.0400,17818
5,6,960418.2900,18757
6,7,1029110.6800,19997
7,8,929383.9400,18839
8,9,952839.5600,18699
9,10,1022479.1200,20273


In [30]:
# First aggregate the 2014 data by week.


aggregations2 = {
   "Item_Value": "sum",
   "Key": pd.Series.nunique
}

GroupWeek2014 = df2014.groupby(by = ["Week"], as_index = False).agg(aggregations2).rename(columns = {"Item_Value": "Weekly_Total_Sales", "Key": "Weekly_Total_Trips"})
GroupWeek2014

,Week,Weekly_Total_Sales,Weekly_Total_Trips
0,1,1086751.4700,19132
1,2,965710.1200,18223
2,3,968321.2200,18176
3,4,1030003.0700,18632
4,5,1008106.6700,18105
5,6,1016374.0800,18635
6,7,1018504.2300,18665
7,8,994294.4900,18466
8,9,1002250.1800,18578
9,10,1054961.6258,19555


In [36]:
# First aggregate the 2015 data by week.


aggregations3 = {
   "Item_Value": "sum",
   "Key": pd.Series.nunique
}

GroupWeek2015 = df2015.groupby(by = ["Week"], as_index = False).agg(aggregations3).rename(columns = {"Item_Value": "Weekly_Total_Sales", "Key": "Weekly_Total_Trips"})
GroupWeek2015

,Week,Weekly_Total_Sales,Weekly_Total_Trips
0,1,402118.1600,7120
1,2,1009602.8300,18043
2,3,973376.0700,17522
3,5,94983.6600,1748
4,6,985829.4900,18418
5,7,1084778.6700,18898
6,8,1041714.3100,18783
7,9,1023942.9100,18458
8,10,1064134.7900,19144
9,11,1037017.9200,18354


In [32]:
# Assigning the new week values to 2014 data.

GroupWeek2014['Week'] = range(53, 53+len(GroupWeek2014))
GroupWeek2014

,Week,Weekly_Total_Sales,Weekly_Total_Trips
0,53,1086751.4700,19132
1,54,965710.1200,18223
2,55,968321.2200,18176
3,56,1030003.0700,18632
4,57,1008106.6700,18105
5,58,1016374.0800,18635
6,59,1018504.2300,18665
7,60,994294.4900,18466
8,61,1002250.1800,18578
9,62,1054961.6258,19555


In [45]:
# Assigning the new week values to 2015 data.

GW2015_1 = GroupWeek2015.iloc[:3]
GW2015_2 = GroupWeek2015.iloc[3:52]

GW2015_1['Week'] = range(105, 105+len(GW2015_1))
GW2015_2['Week'] = range(109, 109+len(GW2015_2))
GroupWeek2015 = GW2015_1.append(GW2015_2, ignore_index=True)
GroupWeek2015

,Week,Weekly_Total_Sales,Weekly_Total_Trips
0,105,402118.1600,7120
1,106,1009602.8300,18043
2,107,973376.0700,17522
3,109,94983.6600,1748
4,110,985829.4900,18418
5,111,1084778.6700,18898
6,112,1041714.3100,18783
7,113,1023942.9100,18458
8,114,1064134.7900,19144
9,115,1037017.9200,18354


In [49]:
# Append all data from 3 years to one. That data will be grouped weekly data.

sample = GroupWeek2013.append(GroupWeek2014, ignore_index=True)
WeeklyTotal = sample.append(GroupWeek2015, ignore_index=True)
WeeklyTotal

,Week,Weekly_Total_Sales,Weekly_Total_Trips
0,1,1069091.6600,19645
1,2,920332.1500,18371
2,3,940726.4600,18176
3,4,978813.5100,18724
4,5,942373.0400,17818
...,...,...,...
151,153,1099724.4300,19318
152,154,1048967.4600,19078
153,155,1197133.1700,20471
154,156,1216750.0100,18407


In [ ]:
# Calculate statistics of distribution

pd.options.mode.chained_assignment = None  # default='warn'

GroupWeek["Weekly_Total_Sales"] = pd.to_numeric(GroupWeek["Weekly_Total_Sales"]).round(2)

# Mean for the distribution
print("Mean Weekly Total Sales: $" + str(round(myStats.mean(GroupWeek["Weekly_Total_Sales"]), 2)))

# Median for the distribution
print("Median Weekly Total Sales: $" + str(round(myStats.median(GroupWeek["Weekly_Total_Sales"]), 2)))

# Standard Deviation for the distribution
print("Volatility Weekly Total Sales: $" + str(round(myStats.stdev(GroupWeek["Weekly_Total_Sales"]), 2)))

In [ ]:
#Plot weekly sales distribution.

plt.figure(figsize = (12, 8))
sb.histplot(data=GroupWeek, x="Weekly_Total_Sales", bins = 20, kde=True)
plt.axvline(round(GroupWeek["Weekly_Total_Sales"].quantile(.05), 2), color = "red", linestyle = "-.", label = "Value at Risk at 95% confidence level")
plt.axvline(round(GroupWeek["Weekly_Total_Sales"].quantile(.01), 2), color = "orange", linestyle = "-.", label = "Value at Risk at 99% confidence level")
plt.xlabel("Weekly Total Sales (in Dollars)")
plt.ylabel("Probability Density")
plt.title("Distribution of Weekly Total Sales for 2013-2015")
plt.legend()
plt.ticklabel_format(style='plain')
plt.show()

In [ ]:
# Calculating Value at Risk for Weekly Total Sales at 5% and 1% significance level:

print("Value at Risk for Daily Sales at 95% confidence level: $" + str(round(GroupWeek["Weekly_Total_Sales"].quantile(.05), 2)))
print("Value at Risk for Daily Sales at 99% confidence level: $" + str(round(GroupWeek["Weekly_Total_Sales"].quantile(.01), 2)))